In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:

# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # self.dataset = load_dataset('lansinuote/ChnSentiCorp', split=split)
        self.dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        print(self.dataset['text'][0])
        print(self.dataset['label'][0])
        # self.dataset = load_from_disk('./data/ChnSentiCorp')
        # print(self.dataset)
        # self.dataset = datasets.fetch_openml('./data/ChnSentiCorp', split=split)
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        label = self.dataset['label'][i]
        return text, label

In [3]:

dataset = Dataset('train') #['train']

# print(dataset['train'][0:10])
# print(dataset['train'][1])

dataset

选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
1


In [ ]:
dataset[0]

## 加载模型

In [4]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
print('token: ', token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)
# print('pretrained ', pretrained)

token:  BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 定义 collate_fn 方法

In [5]:
def collate_fn(data):
    # print(type(data))
    # print('data数据: %s' % data)
    # print('data长度: ', len(data))

    
    # 取每个评论的第一个字符作为数据, 第二作为标签
    # 猫和老鼠的DVD,我在当当网已买过10余次了。除了做为礼物送给亲朋好有的孩子外，...
    
    sents =  [i[0] for i in data] # 猫 ...  16个
    labels = [i[1] for i in data] # 和 ...   16个
    
    #print('sents: %s sents长度: %s' % (sents, len(sents)))
    #print('labels: %s' % labels)    
    
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True
                                  )
    
    # dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
    # print('编码后的date: ', data.keys()) 
    
    #print('input_ids状态: ', data['input_ids'].shape) [16, 500]
    #print('token_type_ids状态: ', data['token_type_ids'].shape) [16, 500]
    #print('attention_mask状态: ', data['attention_mask'].shape) [16, 500]
    
    # 打印
    # print(data['input_ids'])
    
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [6]:

# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)


for i, values in enumerate(loader):
    input_ids, attention_mask, token_type_ids, labels = values
    
    print('')
    # print(i, values)
    print(i, input_ids.shape, 
          attention_mask.shape, 
          token_type_ids.shape, 
          labels
         )
    print('')
    if i >= 2:
        break
    

print(len(loader))


0 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]


1 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


2 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]

600


## 模型训练

In [7]:
# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

    
#模型试算
#out = pretrained(input_ids=input_ids,
#           attention_mask=attention_mask,
#           token_type_ids=token_type_ids)

#out.last_hidden_state.shape

In [8]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()


#model(input_ids=input_ids,
#      attention_mask=attention_mask,
#      token_type_ids=token_type_ids
#     ).shape


model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

## 模型训练

In [9]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
loss = torch.nn.CrossEntropyLoss()


model.train()


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    
    # print('input_ids.shape: %s, labels.shape: %s \n' % (input_ids.shape, labels.shape))
    # print('input_ids: %s, labels: %s' % (input_ids, labels))
    # print("")
    
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = torch.tensor(labels).to(device)
        
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    # print('')
    # print('out状态: ', out, out.shape)
    # print('labels状态: ', labels, labels)
    
    # 梯度下降
    l = loss(out, labels)
    
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    

    if i % 5 == 0:
        out = out.cpu()
        labels = labels.cpu()
            
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        print(i, l.item(), accuracy)

    if i == 300:
        break

/home/mylady/.virtualenvs/dl-pytorch/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.7648610472679138 0.375
5 0.657814621925354 0.5625
10 0.6514464020729065 0.625
15 0.7000013589859009 0.5625
20 0.6312711238861084 0.6875
25 0.586837649345398 0.9375
30 0.6226174235343933 0.6875
35 0.5676578879356384 0.9375
40 0.5505883693695068 0.8125
45 0.5024958848953247 0.9375
50 0.45495858788490295 1.0
55 0.5161272287368774 0.875
60 0.4778011441230774 1.0
65 0.5250521898269653 0.8125
70 0.528820812702179 0.75
75 0.4680573046207428 0.9375
80 0.50459885597229 0.875
85 0.5281070470809937 0.8125
90 0.5644564032554626 0.875
95 0.47969406843185425 0.8125
100 0.4452677071094513 0.875
105 0.46879222989082336 0.875
110 0.48305755853652954 0.8125
115 0.5238274335861206 0.875
120 0.4700734317302704 0.9375
125 0.5667294859886169 0.6875
130 0.48260486125946045 0.875
135 0.50947505235672 0.75
140 0.4635060131549835 0.875
145 0.42668217420578003 0.9375
150 0.49074429273605347 0.8125
155 0.4279915690422058 0.9375
160 0.42926204204559326 0.9375
165 0.4293033182621002 0.9375
170 0.407031655311584

## 测试

In [60]:

def test_res():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i > 100:
            break
            
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = torch.tensor(labels).to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.cpu()
        labels = labels.cpu()
        out = out.argmax(dim=1)
        
        # print('text: %s, 预测值: %s, 真实值: %s' % (input_ids, out, labels))
        
        correct += (out == labels).sum().item()
        total += len(labels)
        
        print('当前批次, acc: %s' % (out.sum() / (out == labels).sum()))

    print('acc: ', correct / total)


In [61]:

# 测试结果
test_res()

這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~ 中餐廳的廣東點心不太好吃~~要改善之~~~~但算價錢平宜~~可接受~~ 西餐廳格調都很好~~但吃的味道一般且令人等得太耐了~~要改善之~~
1
当前批次, acc: tensor(0.6429)
当前批次, acc: tensor(0.5385)
当前批次, acc: tensor(0.4615)
当前批次, acc: tensor(0.4815)
当前批次, acc: tensor(0.5862)
当前批次, acc: tensor(0.4138)
当前批次, acc: tensor(0.5517)
当前批次, acc: tensor(0.3600)
当前批次, acc: tensor(0.4667)
当前批次, acc: tensor(0.5357)
当前批次, acc: tensor(0.3913)
当前批次, acc: tensor(0.5185)
当前批次, acc: tensor(0.6207)
当前批次, acc: tensor(0.4286)
当前批次, acc: tensor(0.5667)
当前批次, acc: tensor(0.4286)
当前批次, acc: tensor(0.4400)
当前批次, acc: tensor(0.4667)
当前批次, acc: tensor(0.5517)
当前批次, acc: tensor(0.3448)
当前批次, acc: tensor(0.6429)
当前批次, acc: tensor(0.6538)
当前批次, acc: tensor(0.7200)
当前批次, acc: tensor(0.6000)
当前批次, acc: tensor(0.3333)
当前批次, acc: tensor(0.4516)
当前批次, acc: tensor(0.7692)
当前批次, acc: tensor(0.6429)
当前批次, acc: tensor(0.5517)
当前批次, acc: tensor(0.5517)
当前批次, acc: tensor(0.5556)
当前批次, acc: tensor(0.6000)
当前批次, acc: tensor(0.4828)
当前批次, acc: tensor(0.633

In [ ]:
dataset

## 计算后的模型保存

In [12]:
# 保存
torch.save(model.state_dict(), 
           'chinese_class_mission_2023_4_10.pt') # 推荐的文件后缀名是pt或pth

In [55]:
# 封装检测类

def str_felling_detect(model, str_sents):
    

    out = token.encode_plus(str_sents,
                            # 当句子长度大于max_length时,截断
                            truncation=True,
                            # 一律补pad到 max_length长度
                            padding='max_length',
                            add_special_tokens=True,
                            max_length=500,

                            #可取值tf,pt,np,默认为返回list
                            return_tensors=None,
                            #返回token_type_ids
                            return_token_type_ids=True,
                            #返回attention_mask
                            return_attention_mask=True,
                            #返回special_tokens_mask 特殊符号标识
                            return_special_tokens_mask=True,

                            #返回offset_mapping 标识每个词的起止位置,
                            # 这个参数只能BertTokenizerFast使用
                            #return_offsets_mapping=True,

                            #返回length 标识长度
                            return_length=True
                           )
    
    input_ids = torch.tensor([out['input_ids']]).to(device)
    attention_mask = torch.tensor([out['attention_mask']]).to(device)
    token_type_ids = torch.tensor([out['token_type_ids']]).to(device)

    out_test = model(input_ids=input_ids,
                     attention_mask=attention_mask,                 
                     token_type_ids=token_type_ids
                    )
    print(out_test)
    
    # print(out_test.argmax(dim=1).cpu())  # tensor([1])
    out_res = out_test.argmax(dim=1).cpu().item()
    # print(out_res, type(out_res))  # <class 'int'>
    
    if out_res == 1:
        print('1 正面情感输出')
    elif out_res == 0:
        print('0 负面情感输出')
    else:
        print('中性句子')

    return out_res

In [59]:
str_1 = '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁' \
       '泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'

str_2 = """1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办."""

str_3 = '房间太小。其他的都一般。。。。。。。。。'

str_4 = "呵呵，虽然表皮看上去不错很精致，但是我还是能看得出来是盗的。但是里面的内容真的不错，我妈爱看，我自己也学着找一些穴位。"

str_5 ="地理位置佳，在市中心。酒店服务好、早餐品种丰富。我住的商务数码房电脑宽带速度满意,房间还算干净，离湖南路小吃街近"


for str_sents in [str_1, str_2, str_3, str_4, str_5]:
    str_felling_detect(model, str_sents)
    print("")

tensor([[0.0763, 0.9237]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
1 正面情感输出

tensor([[0.9981, 0.0019]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

tensor([[0.9577, 0.0423]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

tensor([[0.8644, 0.1356]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

tensor([[0.0068, 0.9932]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
1 正面情感输出

